In [ ]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import collections
import random 
from import_ipynb import *

import Tokenization 
import tensorflow as tf 

# gán biến flag bằng một module trong tensorflow  
# cho phép tạo và xử lý các tham số dòng lệnh 
flags = tf.flags


FLAGS = flags.FLAGS

In [ ]:
# Định nghĩa một tham số đầu vào dạng string = None và mô tả của nó.
flags.DEFINE_string("input_file", None,
                    "Input raw text file (or comma-separated list of files).")
# Định nghĩa một đầu ra tham số dạng string = None và mô tả. 
flags.DEFINE_string(
    "output_file", None , 
    "Output TF exemple file (or comma-separated list of files)."
)

# định nghãi mộtu tham số dáng tring vocab_file  = None và mô tả của nó .
flags.DEFINE_string("vocab_file", None, 
                    "The vocabulary file thar the Bert model was trained on.")

# Định nghĩa một tham số dạng bool là do_lower_case  = True và mô tả của nó
flags.DEFINE_bool(
    "do_lower_case", True, 
    "Whether to lower case the input text. Should be True for uncased"
    "models and False for cased models."
)
# Định nghĩa một tham số do_whole_word_mask dạng boolen  gán = False và mô tả của nó . 
flags.DEFINE_bool(
    "do_whole_word_mask", False, 
    "Whether to use whole word masking rather than per-WordPiece masking."
)

# Tương tự định nghĩa một tham số dạng integer là max_seq_length = 128  
flags.DEFINE_integer("max_seqg_length", 128, "Maximum sequence length.")
# Tương tự như trên với biến max_prediction_per_seq 
flags.DEFINE_integer("max_predictions_per_seq", 20, 
                     "Maximum number if masked LM predictions per sequence.")

flags.DEFINE_integer("dupe_factor", 10, 
                     "Probability of creating sequences wwhich are shorter tha the"
                     "maximum length.")


In [ ]:
# xây dựng lớp trường hợp huấn luyện 
class TrainingInstance(object):
    """A single traing instance (sentence pair).
        Một trường hợp huấn luyện duy nhất . 
    """
    # Thiết lập phương thức khởi tạo 
    def __init__(self, tokens, segment_ids , masked_lm_positions, masked_lm_labels, 
                is_random_next):
        self.tokens = tokens 
        self.segment_ids = segment_ids 
        self.masked_lm_positions = masked_lm_positions 
        self.masked_lm_labels = masked_lm_labels 
        self.is_random_next = is_random_next

    # Thiết lập phương thức def _str_ cho lớp trainingInstance 
    # lớp này được sử dụng để lưu trữ các thông tin liên quan đến một mẫu huấn luyện 
    # cho mô hình bert 
    def __str__(self):
        #  Khởi tạo một biến s = 1 chuỗi rỗng 
        # để lưu trữ các thuộc tính của thể hiện huấn luyện 
        s = ""
        # cộng biến s với 1 chuỗi giá trị tokens được mã hóa , cách nhau bởi dấu cách 
        # hàm tokenization.Printable_text chuyển đổi một token thành một chuỗi có thể đọc .
        s += "tokens: %s\n" % (" ".join(
            [Tokenization.printable_text(x) for x in self.tokens]
        ))
        # tuwowng tujw nhuw trên thêm vào một chuỗi s một dòng chứa các segment_ids , cách nhau
        # bởi dấu cách, sử dụng str chuyển số nguyên thành  1 chuỗi vd :  nếu self.segment_ids là [0, 0, 0, 0, 0],
        #  thì dòng này sẽ thêm vào s chuỗi "segment_ids: 0 0 0 0 0\n".
        s += "segment_ids: %s \n" %(" ".join([str(x) for x in self.segment_ids]))
        # Thêm vào chuỗi s một dòng chứa các giá trị của thuộc tính is_trandom_next .
        # strt được ngầm định chuyển một giá trị bool thành một chuỗi 
        s += "is_random_next: %s \n" % self.is_random_next 
        # Thêm vào chuỗi s một dòng chứa các vị trí bị che đi , cách nhau bỏi dấu cách . 
        # ví dụ : Ví dụ: nếu self.masked_lm_positions là [3], 
        # thì dòng này sẽ thêm vào s chuỗi "masked_lm_positions: 3\n"
        s += "masked_lm_positions: %s \n" % (" ".join([str(x) for x in self.masked_lm_positions]))
        # Thêm vào chuỗi s một dòng chứa các nhãn đúng cho các vị trí bị che đi, cách nhau bởi dấu cách
        #  Ví dụ: nếu self.masked_lm_labels là [2024], 
        # thì dòng này sẽ thêm vào s chuỗi "masked_lm_labels: a\n"
        s += "masked_lm_labels: %s \n" % (" ".join([Tokenization.printable_text(x) for x in self.masked_lm_labels]))
        # thêm vào chuỗi s một dòng trống để tách biệt các thể hiện huấn luyện khác nhau .
        s += "\n"
        return s 
    def __repr__(self):
        return self.__str__()

In [ ]:
# xây dựng phương thức create_int_feature 
# tạo ra các đặc trưng định dạng int 
def create_int_feature(values):
    # nhận vào một danh sách các số nguyên và trả về một đối tượng tf.train.Feature 
    # là một lớp được sử dụng để lưu trữ một giá trd dơn hoặc nhiều giá trị cùng kiểu 
    # dữ liệu 
    feature = tf.train.Feature(int64_list = tf.train.Int64List(value=list(values))
                               )
    return feature 


# xây dựng phương thức create_infloat_feature 
# tạo ra các đặc trưng định dạng float
def create_float_feature(values):
  #  sử dụng float_list, là một lớp được sử dụng để lưu trữ một danh sách các số thực 32 bit.
  feature = tf.train.Feature(float_list=tf.train.FloatList(value=list(values)))
  return feature

# xây dụng phương thức write instance to example files
# đê tạo ra các file mẫu tf từ lớp trường hợp huấn luyện  
def write_instance_to_example_files(
        instances , tokenizer , max_seq_length , max_predictions_per_seq , 
        output_files
):
    """Create TF example files from 'TrainingInstance's."""
    # tạo một danh sách writers để lưu trữ các bản gi 
    writers = []
    # duyêt nqua 1 danh sách các file đầu ra
    for output_file in output_files:
        # thêm các bản ghi vào danh sách viết 
        writers.append(tf.python_io.TFRecorWriter(output_file))

     # khowir taij mootj biến số nguyên để theo dõi chỉ số của đối tượng TFRecordWriter 
     # hiện tại trong danh sách writes . 
    writer_index =0 
    # tạo một biến số nguyên để theo dõi số lượng thể hiện huấn luyện đã được ghi vào các 
    # tệp đầu ra 
    total_written = 0

    # duyệt nqnua một danh sách các thể hiện huấn luyện và lấy ra chỉ số và giá trị của mỗi
    # thể hiện 
    for (inst_index , instance) in enumerate(instances):
        # chuyển đổi 1 danh sách các tokens của thể hiện huấn luyện thành một danh sách các
        # số nguyên biểu diễn mã hóa của chúng 
        input_ids = tokenizer.convert_tokens_to_ids(instance.tokens)
        # tạo một danh sách có cùng độ dài với dnh sách input_ids và gán gái trị 1 cho 
        # tất cả các phần tử . Đây là một mătj nạ để chỉ ra các vị trí có chứa tokens hợp lệ 
        # trong đầu vào . 
        input_mask = [1] * len(input_mask)
        # sao chép danh sách các segment_ids của thể hiện huấn luyện thành 1 danh sách mới . 
        # Đây là một dnah sách các số nguyên chỉ ra thuộc về đoạn nào trong một cặp văn bản đầu vào 
        segment_ids = list(instance.segment_ids)

        # kiểm tra xem độ dìa của danh sa hs input_ids đó có nhỏ hơn hoặc bằng độ dài tối đa của 
        # một chuỗi đầu vào hay không. Nếu không dừng trương trình và báo lỗ 
        assert len(input_ids) <= max_seq_length

        # trong khi độ dài input_ids  < max_seq length 
        while len(input_ids) < max_seq_length: 
            # thêm các giá trị = 0 vào 3 danh sách 
            input_ids.append(0)
            input_mask.append(0)
            segment_ids.append(0)

        # kiểm tra xem độ dài của input_ids , input_mask , segment_ids có 
        # bằng với max_seq_length không 
        assert len(input_ids) == max_seq_length
        assert len(input_mask) == max_seq_length
        assert len(segment_ids) == max_seq_length

        # sao chép danh sách các vị trí bị tre đi của thể hiện huấn luyện thành một danh sách mới 
        # đây là danh sách các số nguyên chỉ ra các vị trí của tokens bị che để dự đoán bằng mô hình ngôn ngữ 
        masked_lm_positions = list(instance.masked_lm_possitions)
        # chuyển đổi danh sách các nhãn đúng cho các vị trí bị che đi của thể hiện hiaasn luyện thành 
        # một danh sách các số ngyên biểu diễn mã hóa chúng 
        masked_lm_ids = tokenizer.convert_tokens_to_ids(instance.mask_lm_labels)
        # tạo một danh sách có cùng độ dài với masked_lm_ids và gán giá trị 1.0 cho tất cả các 
        # phần tử. Đây là một danh sách các số thực biểu diễn trọng số của các vị trị bị tre đi 
        masked_lm_weights =[1.0] * len(masked_lm_ids)

        # lặp lại cho đến khi len(masked_lm_positions) < max_predictions_per_seq 
        # tức alf đến khi độ dài của danh sách masked_lm_positions bằng với số lượng dự đoán 
        # tối đa của một chuỗi đầu vào. 
        while len(masked_lm_positions) < max_predictions_per_seq:
            masked_lm_positions.append(0)
            masked_lm_ids.append(0)
            masked_lm_weights.append(0.0)

        # gán cho biến next_sentence_label giá trị = 1 nếu thuộc tính is_random_next của thể hiện 
        # huấn luyện là True ,ngược lại là 0. đây là 1 giá trị nguyên chỉ ra xem liệu văn bản thứ 2
        # có được chọn ngẫu nhiên hay không . 
        next_sentence_label = 1 if instance.is_random_next else 0

        # khởi tạo một đối tượng OrderedDict để lưu trữ các đặc trưng của thể hiện huấn luyện . 
        # sử dụng để tạo ra một từ điển có thứ tự 
        features = collections.OrderedDict()
        features["input_ids"] = create_int_feature(input_ids)
        features["input_mask"] = create_int_feature(input_mask)
        features["segment_ids"] = create_int_feature(segment_ids)
        features["masked_lm_positions"] = create_int_feature(masked_lm_positions)
        features["masked_lm_ids"] = create_int_feature(masked_lm_ids)
        features["masked_lm_weights"] = create_float_feature(masked_lm_weights)
        features["next_sentence_labels"] = create_int_feature([next_sentence_label])

        # khởi tạo biến tf_example  bằng các đặc trưng của thể hiện huấn luyện được xử ký 
        # tạo một đối tuowngj tf.train.Examople là một lớp được sử dụng để lưu trữ một bản ghi
        # TFRecord từ đối tượng features
        tf_example = tf.train.Example(features=tf.train.Features(features=features))

        # ghi đối tượng tf_example vào tập đầu ra tương ứng với chỉ số wi=rite_index trong danh sách 
        # đối tượng tf_example được chuyển đổi thành một chuỗi nhị phân với SerializeToString()
        writers[writer_index].write(tf_example.SerializeToString())
        # caapj nhật giá trị của biến writer_index bằng cách tăng nó lên 1 và lấy phần dư chi độ dài 
        # của danh sách write nhằm mục đích phân bổ các thể hiện huấn luyện đều cho cácteepj đầu ra khác nhau 
        writer_index = (writer_index + 1) % len(writers)

        total_written += 1

        # kiểm tra xem inst_index  có nhỏ hơn 20 tức chỉ số index theo gái trị trong danh sách 
        # các thể hiện 
        if inst_index < 20:
            # ghi một thông báo vào nhật ký nhằm đánh dấu một thể hiện huấn luyện 
            tf.logging.infor("*** Example ***")
            # ghi một thông báo vào nhật ký chứa danh sách các tokens của thể hiện huấn luyện 
            # cách nhau bởi dấu cách . và chuyển dổi một tokens thành 1 chuỗi đọc được 
            tf.logging.info("tokens: %s" % " ".join([
                Tokenization.printable_text(x) for x in instance.tokens]))
            # duyệt qua các khóa của đối tượng features là một OrderedDict chứa các đặc trưng của thể hiện huấn luyện.
            for feature_name in features.keys():
                # láy ra giá trị đặc trưng ứng với khóa feature_name.
                # OrderedDict chứa các đặc trưng của thể hiện huấn luyện.
                feature = features[feature_name]
                # tạo 1 danh sách rỗng để luuw trữ các giá trị 
                values = []
                # kiểm tra xem các giá trị trong dah sách giá trị đặc trưng của thể hiện huấn luyện 
                # có thuộc kiểu int.64 
                if feature.int64_list.value: 
                    # gán danh sách values = các giá trị dạng list_int trong feature 
                    alues = feature.int64_list.value
                # trường hợp là kiểu float ta làm tương tự 
                elif feature.float_list.value:
                    # và cũng gán dnah sách value
                    values = feature.float_list.value
                # ghi một thông báo vào nhật ký chứa một danh sachs các tên đặc trưng trong features và 
                # giá trị của nó được cách nhau bởi khoảng trắng. 
                tf.logging.info(
                        "%s: %s" % (feature_name, " ".join([str(x) for x in values])))
                
        # duyệt qua danh sách các bản ghi 
        for writer in writers:
            # đóng các bản ghi đó lại 
            writer.close()

        # ghi một mã thông báo vào nhật ký cho biết tổng số bản ghi đã được đọc 
        tf.logging.info("Wrote %d total instances", total_written)



In [1]:
# Xây dựng phương thức khởi tạo các trường hợp huấn luyện 
def create_training_instances(input_files , tokenizer, max_seq_length , 
        dupe_factor , short_seq_prob, masked_lm_prob , 
        max_predictions_per_seq, rng                      
            ):
    """Create "TrainingInstance"s from raw text."""
    all_documents = [[]]
    # Input file format: 
    # (1) One sentence per lien. These should ideally be actual sentences , not entire paragraphs 
    #   or arbitrary spans of text . (Because we use the sentences bounderies for the " next sentences
    #   prediction task").

        # Mỗi dòng một câu . Lý tưởng nhất đây phải là những câu thực tế . Không phải toàn bộ đoạn văn 
        # các đoạn văn bản tùy ý . (Bởi vì sử dụng ranh giới câu cho nhiệm vụ dự đoán câu tiếp theo)
    # (2) Blank lines between documents. Document bounderies are needed so that the " next sentences prediction" task 
    # doesn't span between documents . 

        # Dòng Trống giữa các tài liệu , cần có danh giới tài liệu để có thể dự đoán câu tiếp theo 
        # không trải dài các dữ liệu 

    # duyệt qua dnah sách các file đầu vào 
    for input_file in input_files: 
        # mở các file đầu vào từ danh sachs các file đầu vào 
        with tf.gfile.GFile(input_file , "r") as reader:
            # với điều kiện = True 
            while True : 
                # dọc các dòng văn bản có trong file dưới định dnagj unicode 
                line = Tokenization.convert_to_unicode(reader.readline())
                # nếu không tồn tại bất kỳ giá trị nào trong line 
                if not line: 
                    # thoát khỏi vòng lặp 
                    break 
                # trường hợp còn lại gán line = line.strip để chuẩn hóa tachs các chuỗi rồng 
                line = line.strip()

                # các dòng trống sẽ được sử dụng làm dấu phân cách tài liệu 
                # kiểm tra xem line có tồn tại 
                if not line :
                    # thêm một khoẳng trắng vào danh sách chưa tất cả tài liệu 
                    all_documents.append([])
                # Thực hiện ãm hóa tiêu chuẩn các dòng văn bản bằng Tokenizer.tokenize
                tokens = tokenizer.tokenize(line)
                # nếu có tồn tại tokens 
                if tokens: 
                    # thêm token vào vị trí của cùng của danh sách chưa tài liệu hiện tại 
                    all_documents[-1].append(tokens)

        # xóa đi các tài liệu trống 
        # tạo một danh sách chưa tài liệu mới bằng cách lặp qua danh sách ban đầu và lưu các 
        # tài liệu không rỗng có trong danh sách vào danh sách mới
        all_documents = [x for x in all_documents if x]
        # xáo trộn các tài liệu trong danh sách 
        rng.shuffle(all_documents)

        # lấy ra danh scahs ácc từ trong từ điển của tokenizer à lưu vào biến vocab_word 
        vocab_word = list(tokenizer.vocab.keys())
        instances = []

        # duyệt qua dupe_factor lần mỗi lần duyêth qua tất cả cac tài liệu trong all_documents 
        for _ in range(dupe_factor):
            for document_text in range(len(all_documents)):
                # sử dụng hàm extend để thêm các tài liệu vừa tạo từ một tài liệu vào danh sách 
                # instances 
                instances.extend(
                    create_instances_from_document(
                        all_documents , document_text, max_seq_length , short_seq_prob, 
                        masked_lm_prob, max_predictions_per_seq , vocab_word, rng
                    )
                )

        # Trộn các thể hiện của instances và trả về kết quả của đoạn mã 
        rng.shuffle(instances)
        return instances 
    

# Xây dựng phương thức khởi tại các trường hợp từ tài liệu 
def create_instances_from_document(
        all_documents , document_index, max_seq_length , short_seq_prob, 
        masked_lm_prob , max_predictions_per_seq , vocab_words , rng
):
    # Tạo ra các trường hợp huấn luyện cho một tài liệu đơn độc . 
    # tạo một tài liệu document = cách ánh xạ các chỉ số index_document cho danh sách 
    # chứa tất cả tìa liệu 
    document = all_documents[document_index]

    # tạo một biến số lượng tokens trong 1 câu hay một chuỗi đầu vào 
    # bằng cách trừ đi 3 chỉ số là các tokens đặc biệt 
    max_num_tokens = max_seq_length - 3

    # Chúng ta thường muốn phủ lên toàn bộ chuỗi vì dù sao chúng at cũng đệm vào max_seq_length 
    # vì vậy các chuỗi ngắn thường  lãng phí tính toán 
    # khởi tạo một biến mục tiêu là độ dài mục tiêu cho số lượng tokens của 1 đoạn đầu vào 
    target_seq_length = max_num_tokens

    # sử dụng hàm random.random để tạo ra 1 giá trị [0->1] và so sánh nó với 
    # xác suất sinh ra chuỗi ngắn và kiểm tra xem giá trị ngẫu nhiên cod nhỏ hơn 
    # xác suất sinh ra chuỗi ngắn không 
    if rng.random() < short_seq_prob: 
        # gán biến độ dài cho đoạn văn bản mục tiêu bằng giá trị ngẫu nhiên 
        # từ 2 đến số tokens tối đa của một chuỗi văn bản 
        target_seq_length = rng.randint(2 , max_num_tokens)

    # tạo da nh sách instances để lưu trữ các thể hiện 
    instances = []
    # một danh sách current_chunk để lưu trữ các đoạn văn bản hiện tại 
    # mỗi đoạn văn bản là dnah scahs các câu đã được mã hóa thành dnah sách các 
    # mã thông báo 
    current_chunk = []
    # một danh scahs current để lưu trữ độ dài hiện tại của đoạn văn bản đang xử lý 
    # từ currrent chunk 
    current_length = []
    # và biến i đếm số đoạn văn bản trong current_chunk 
    i = 0 
    # kiểm tra xem số lượng đoạn hiện tại có nhỏ hơn số đoạn trong tài liệu 
    while i < len(document):
        # gán biến segment đoạn văn bản hiện tại bằng đoạn văn bản thứ i trong document 
        segment = document[i]
        # và sau đó thêm đoạn văn bản hiện tại vào danh sách lưu trữ current_chunk 
        current_chunk.append(segment)
        # cộng vào current_length một tham số là độ dài của đoạn hiện tại
        current_length += len(segment)
        # kiểm tra xem số đoạn hiện tại trong current_chunk có bằng với số đoạn trong tài liệu chưa 
        # hay độ dài của các đoạn hiện tại , current_length >=  độ dài của đoạn văn bản mục tiêu 
        if i == len(document)  or current_length >= target_seq_length:
            # và kiểm tra xem current_chunk có tồn tại không 
            if current_chunk: 
                # tạo một biến a_end là giá trị cho biết số lượng đoạn đã được sử dụng làm câu A để dự đoán câu b 
                # từ dnah sách chứa các đoạn hiện tại 
                a_end = 1 
                # ssau đó kiểm tra xem số lượng đoạn hiện tại từ danh sách current chunk có 
                # lớn hơn = 2 không 
                if len(current_chunk) >= 2 : 
                    # đặt lại a_end = môth giá trị ngẫu nhiên từ 1 đến số lượng đoạn hiện tại - 1
                    a_end = rng.randint(1 , len(current_chunk) - 1)

                # tạo một danh sách tokens để lưu trữ tokens của câu a  
                tokens_a = []
                # lặp qua một danh sách từ 0-> a_end cho biết ố đoạn hiện tại trong current_chunk 
                for j in range(a_end):
                    # thêm danh cách tokens chứa trong mỗi đoạn từ 0 đến a_end trong danh sách current_chunk 
                    # sử dụng hàm extend để nối các đoạn lại với nhau 
                    tokens_a.extend(current_chunk[j])
                
                # Tạo một dnah sách tokenS_b để lưu trữ các tokens của câu B 
                tokens_b = []
                # kiểm tra xem current_chunk == 1 tức là chỉ chứa duy nhất 1 đoạn 
                # hoặc xác suất ngẫu nhiên < 0.5
                # khởi tạo một biến là random next = False 
                is_random_next = False
                if len(current_chunk) == 1 or rng.random() < 0.5:
                    # đặt biến is_random_next = True tức là B là một câu theo sau câu A 
                    # trong document Điều này để taọ nhãn cho Tác vụ NSP 
                    is_random_next = True 
                    # khởi tạo một giá trị là độ dài mục tiêu cho B bằng độ dài đoạn mục tiêu 
                    # trừ độ dài của dnah sách tokens chứa câu a 
                    target_b_length = target_seq_length - len(tokens_a)
                    for _ in range(10):
                        # tạo một biến random_document_index = giá trị ngẫu nhiên từ 0  đến 
                        # số lượng các đoạn văn abnr trong bộ tài liệu document 
                        random_document_index = rng.randint(0 , len(all_documents) -1)
                        # kiểm tra xem chỉ số số chỉ số tài liệu ngẫu nhiên random_document_index 
                        # có khác với chỉ số của bộ tài liệu hay không , nếu khác thì lập tức thoát 
                        # và thực hiện tiếp trương trình .tức là đoạn mã này sẽ chọn 1 bộ tài liệu bắt kỳ nếu trùng chọn lại 
                        # Mục đích của đoạn mã này để đảm bảo rằng tại liệu ngẫu nhiên không 
                        # Trùng với tài liệu hiện tại đoạn mã sẽ kết thúc khi đạt được kết quả là 
                        # một bộ chỉ số khác nhau tức là khi đạt được mục đích 
                        if random_document_index != document_index: 
                            break 

                    # đặt biến random_documnet bằng các giá trị theo chỉ số ngẫu nhiên 
                    # bằng cách ánh xạ chỉ số này vào danh sách chứa các đoạn văn bản từ bộ tài liệu 
                    random_document = all_documents[random_document_index]
                    # Tạo một giá trị random_start chỉ số bắt đầy ngẫu nhiên  là một giá trị bất kỳ 
                    #  khoảng 0 -> len(random_document) - 1
                    random_start = rng.random(0, len(random_document) -1)
                    # duyệt qua một dnah sách các đoạn ngẫu nhiên từ random satrt đến random_document
                    for j in range(random_start, len(random_document) -1):
                        # thêm danh sách các đoạn ngẫu nhiên từ random_document - 1 
                        # vào danh sách tokens_b sử dụng hàm extend để nối các câu này lại '
                        tokens_b.extend(random_document[j])
                        # kiểm tra xem độ dài của danh sách tokens_b > target_b_length 
                        if len(tokens_b) >= target_b_length: 
                            # lập tức thoát khỏi vòng vặp 
                            break 
                    # đếm số lượng các đoạn văn bản không được sử dụng trong danh sách hiện tại 
                    # Giảm giá trị của i đi num_unused_segments để quay lại các đoạn chưa được xử lý
                    num_unsed_segments = len(current_chunk) - a_end
                    i -= num_unsed_segments

                # trường hợp còn lain tức là số đoạn hiện tại củadnah sách current chunk != 1 
                # và giá trị sinh ngẫu nhiên >0.5
                else: 
                    # đặt is_random_next = False tức là B là câu ngẫu nhiên không phải câu theo sau A
                    is_random_next = False 
                    # lặp qua danh ách từ chỉ số a_end đến hết số lượng đoạn hiện tại trong current_chunk 
                    for j in range(a_end, len(current_chunk)):
                        # thêm nó vào dnah sách tokens_b 
                        # tức là thêm các đoạn trong danh sách current_chunk từ đoạn hiện tại đến hết danh 
                        # sách chứa các đoạn hiện tại 
                        tokens_b.extend(current_chunk[j])
                    
                # Sau đó từ danh sách tokens_a , tokens_b ta đư vào phương thức truncate_seq_pair 
                # để cắt ngẵn các cặp câu trong trong danh sách tokens_a , tokens_b nếu tổng số 
                # tokens của cúng quá số tokens tối đa 
                truncate_seq_pair(tokens_a, tokens_b, max_num_tokens, rng)

                # kiểm tra và đưa ra cảnh báo cho độ dài 2 danh sách tokens_a , tokens_b 
                assert len(tokens_a) >= 1 
                assert len(tokens_b) >= 1

                # tạo một dnah sách tokens để lưu trữ các tokens sau khi xử lý 
                tokens = []
                # và một danh sách segment_ids để lưu trữ các nhãn cho các tokens
                # 0 cho câu a và 1 cho câu b 
                segment_ids = []
                # Thêm một tokens [CLS] vào danh sách tokens 
                tokens.append("[CLS]")
                # Tươnh tự thêm 1 token 0 vào dnah sách chứa nhãn 
                segment_ids.append(0)

                # duyệt qua dnah scahs tokens a
                for token in tokens_a:
                    # thêm các tokens vào dnah sách tokens
                    tokens.append(token)
                    # thêm nhãn tương ứng 
                    segment_ids.append(0)
                # thêm nhãn và tokens đặc biệt đánh dấu kết thúc câu 
                tokens.append("[SEP]")
                segment_ids.append(0)
                
                # duyệt qua dnah sách tokens_b 
                for token in tokens_b:
                    # thêm tokens vào dnah scahs tokens
                    tokens.append(token)
                    # thêm nhãn tương ứng vào danh sách 
                    segment_ids.append(1)
                # thêm nhãn và tokens đặc biệt đánh dấu kết thúc câu 
                tokens.append("[SEP]")
                segment_ids.append(1)

                # THỰC HIỆN LỚP TIỀN XỬ LÝ mặt nạ tokens cho mô hình dự đoán 
                (tokens, masked_lm_positions,
                    masked_lm_labels) = create_masked_lm_predictions(
                    tokens, masked_lm_prob, max_predictions_per_seq, vocab_words, rng)
                
                # khởi tạo instance = TrainingIntance 
                # để lưu trữ các thông tin cần thiết cho một thể hiện huấn luyện của kô hình Bert
                # motpj thể hiện huấn luyện là một cặp câu có độ dài ngẫu nhiên 
                instance = TrainingInstance(
                    tokens=tokens,
                    segment_ids=segment_ids,
                    is_random_next=is_random_next,
                    masked_lm_positions=masked_lm_positions,
                    masked_lm_labels=masked_lm_labels)
                # thêm instance vào dnah sách chứa các trường hợp huấn luyện 
                instances.append(instance)
            # đặt lại curent chunk và current_length = rỗng 
            current_chunk = []
            current_length = 0
        # tăng i lên 1 
        i += 1
    # trả về dnah sách chứa các trường hợp được huấn luyện 
    return instances

# Lớp MaskedLmInstance được sử dụng trong đoạn mã tiền xử lý dữ liệu cho mô hình BERT1,
#  để lưu trữ các vị trí và nhãn của các token bị che mặt trong một chuỗi. 
# Các token bị che mặt là những token bị thay thế bằng token [MASK], token ngẫu nhiên,
# hoặc giữ nguyên, với một số xác suất nhất định. Mục đích của việc che mặt các token là 
# để huấn luyện mô hình ngôn ngữ có khả năng hiểu được ngữ cảnh của các token trong một chuỗi1
MaskedLmInstance = collections.namedtuple("MaskedLmInstance",
                                          ["index", "label"])


In [ ]:
# Xây dựng lớp che mặt nạ cho mô hình dự đoán mặt nạ ngôn ngữ Bert 
def create_masked_lm_predictions(tokens, masked_lm_prob, 
                max_predictions_per_seq , vocab_words , rng):
    """Create the predictions for the masked LM objective."""

    # khởi tạo một danh scahs để lưu trữ các chỉ số của tokens ứng viên để có thể che 
    cand_indexes = []
    # duyệt qua danh sách chưa tokens lấy ra chỉ só và giá trị các tokens từ danh sách 
    for (i , token) in enumerate(tokens):
        # kiểm tra xem các tokes có phải là các giá trị đặc biệt không phải thì tiếp tục 
        if token == "[CLS]" or token == "[SEP]":
            continue 

        # kiểm trã xem có nên sử dụng kỹ thuật tre toàn bộ các tokens hay không, 
        # nếu có thì kiểm tra xem tokens hiện tại có bắt đầu bằng ## hay không 
        # ## là 2 token phụ được mã hóa thêm vào 1 từ 
        if (FLAGS.do_whole_word_mask and len(cand_indexes) >= 1 and
        token.startswith("##")):
            # nếu có thêm lập tức chỉ số i vào danh sách chứa chỉ số ứng viên 
            cand_indexes[-1].append(i)
        # trường hợp còn lại tạo cho i là một danh sách mới rồi thêm vào danh sách 
        # chứa chỉ số ứng viên
        else:
            cand_indexes.append([i])
    # xáo trộn các chỉ số ứng viên một cacgs ngẫu nhiên
    rng.shuffle(cand_indexes)
    # tạo một danh sách output_tokens bằng cách sao chép lain danh sách tokens
    output_tokens = list(tokens)
    # tính số lượng tokens cần dự đoán được tính bằng cách lấy giá trị nhỏ nhất giữa max_predictions_per_seq,
    # là một tham số để giới hạn số lượng token tối đa được che trong mỗi câu,
    # và giá trị lớn nhất giữa 1 và số lượng token trong câu nhân với masked_lm_prob, 
    # là một tham số để chỉ xác suất che một token trong câu.
    num_to_predict = min(max_predictions_per_seq,
                       max(1, int(round(len(tokens) * masked_lm_prob))))

    # tạo một danh sách masked_lm để lưu trữ các dối tượng của maskedLmInstance mỗi đối tượng 
    # này chứa thông tin về một tokens bị che đi , bao gồm chỉ số , tokens gốc , tokens thay thế
    masked_lms =[]
    # tạo một danh sách convered_indexes là một tập hợp các chỉ số của ácc tokens đẫ được 
    # che hoặc bỏ qua . để đảm bảo rằng không có tokens nào được tre nhiều hơn 1 lần 
    covered_indexes = set()
    # duyệt qua danh sách các chỉ số ứng viên trong danh sách cand_indexes 
    for index_set in cand_indexes:
        # kiểm tra xem đọ dài danh sách masked_lm đã lớp hơn số tượng tokens cần dự đoán chưa 
        if len(masked_lms) >= num_to_predict:
            # nếu lớn hơn thoát khỏi vòng lặp 
            break 

        # trường hợp nếu như việc thêm mặt nạ toàn từ sẽ vượt quá số lượng dự đoán tối đa 
        # thì cần bỏ qua ứng viên này
        # kiểm tra nếu tổng số phần tử của của masked_lms và chỉ số ứng viên hiện tại 
        # mà vượt quá số lượng cần dự đoán 
        if len(masked_lms) + len(index_set) > num_to_predict: 
            # tiếp tục lặp 
            continue 

        # khởi tạo một biến boolen gọi là is_any_index_convered = False
        # biến này sẽ được sử dụng để kiểm tra xem có bất kỳ tokens nào trong tâp hợp indexes
        # hay danh sách ứng viên đã được tre đi hoặc bỏ qua hay không . 
        is_any_index_convered = False 
        # duyệt qua dnah sách các phần tử trong danh sách các token ứng viên để được tre được 
        # trích xuất từ danh sách ứng viên ban đầu
        for index in index_set: 
            # kiểm tra xem phần tử ứng viên trong danh sách này đã có trong tập convered_indexes 
            # thì gán = True và thoát khỏi vòng lặp . Điều này có nghĩa là tập hơn index_set đã bị trùng với 
            # mọt tập hợp khác và không nên được tre 
            is_any_index_convered = True 
            break 
        
        # kiểm tra xem is_any có  = True không
        if is_any_index_convered :
            # nếu có ta tiếp tục lặp 
            continue 

        # duyệt qua danh sách ứng viên một lần nữa 
        for index in index_set: 
            # thêm chúng avof danh sách convered_indexes 
            # điều này có nghĩa là đánh dấu các tokens index này đã được che hoặc bỏ qua 
            covered_indexes.add(index)

        # khởi tạo một biến gọi là masked_token = None biến này sẽ được sử dụng để lưu trữ 
        # token thay thế cho tokens bị che đi 
        masked_token = None 
        # kiểm tra xem giá trị ngẫu nhiên < 80 % không 
        # tức là ta thay thế 80% số trường hợp bị che đi = MASK 
        if rng.random() < 0.8:
            masked_token = "[MASK]"
        # trường hợp còn lại 
        else: 
            # kiểm tra xem tỷ lệ ngẫu nhiên < 0.5 
            if rng.random() < 0.5: 
                # ta giữ nguyên 10 % số trường hợp ban đầu từ danh sách bị che đi 
                # bằng cách ánh xạ các chỉ số ứng viên vào danh sách tokens và gán cho 
                # danh sách lưu trữ tokens thay thế 
                masked_token = tokens[index]
            # trưuowngf hợp còn lại ta chọn 10% trường hợpn
            # và thay thế nó bởi 1 tokens ngẫu nhiên trong bộ từ điển 
            else:
                masked_token = vocab_words[rng.ranint(0 , len(vocab_words) -1)]

        # gán giá trị của tokens bị thay thế cho token bị che đi , vào vị trí tương ứng trong danh sách 
        # output_tokens . 
        # tokens bị thay thế có thể là cac loại token đặc biệt hoặc token ngẫu nhiên khác . 
        output_tokens[index] = masked_token

        # thay thế một đối tượng maskedLanguageModelInstance vào danh sách masked_lms đối tượng này 
        # chưa chỉ só và tokens gốc của token bị tre đi 
        masked_lms.append(MaskedLmInstance(index=index, label=tokens[index]))
        # kiểm tra xem độ dài của dnah scahs masked_lm có nhỏ hơn số lượng tokens cần dự đoán 
        assert len(masked_lms) <= num_to_predict
        # sắp xếp danh sachs token bị che đi theo thứ tự của chỉ số điều này giúp cho việc dự đoán các 
        # token bị che đi dễ dàng hơn 
        masked_lms = sorted(masked_lms, key=lambda x: x.index)

        # tạo 2 danh scahs mới 
        # 1 danh sách sẽ chứa vị trí accs tokens bị che 
        masked_lm_positions = []
        # và 1 danh scahs chứa nhãn cho các token bị tre
        masked_lm_labels = []
        # duyệt qua dnah sách chưa các token bị tre 
        for p in masked_lms: 
            # thêm vị trí và nhãn của nó vào danh sách tương ứng 
            masked_lm_positions.append(p.index)
            masked_lm_labels.append(p.label)

    # trả về 3 giá trị gòm danh scahs token được tre , vị trí của các token đuwocj tre , 
    # nhãn của ácc tokens được tre 
    return (output_tokens, masked_lm_positions, masked_lm_labels)


# Xây dựng phương thức cắt ngắn các cặp câu trong danh sách tokens a và b 
def truncate_seq_pair(tokens_a , tokens_b , max_num_tokens, rng):
    """cắt ngắn một cặp chuỗi đến một độ dài tối đa"""
    # trong khi điều kiện đúng 
    while True: 
        # tính tổng độ dài chuỗi từ 2 chuỗi câu a , b 
        total_length = len(tokens_a) + len(tokens_b)
        # kiểm tra xem tỏng độ dài có nhỏ hơn số lượng tokens tối đa 
        if total_length <= max_num_tokens:
            # có thì thoát không cần cắt 
            break

        # trường hợp khác ta chọn đoạn alf tokens_a hoặc b
        trunc_tokens = tokens_a if len(tokens_a) > len(tokens_b) else tokens_b
        # kiểm tra xem điều kiện và trả về cảnh báo 
        assert len(trunc_tokens) >= 1

        # tạo ra một giá trị ngẫu nhiên [0 ->1]
        # để ta co thể chọn việc cắt bớt trước hay sau chuỗi văn bản 
        # để tạo tính ngẫu nhiên 
        if rng.random() < 0.5:
            del trunc_tokens[0]
        else:
            trunc_tokens.pop()




def main(_):
    # đặt mức độ ghi nhật ký alf ìnffor để hiển thị thông tị quan trọng trongq úa trình chạy chương trình 
    tf.logging.set_verbosity(tf.logging.INFO)
    # tạo một đối tượng tokenizer với đươngf dâbx đếb file 
    tokenizer = Tokenization.FullTokenizer(
      vocab_file=FLAGS.vocab_file, do_lower_case=FLAGS.do_lower_case)

    # tạo một dnah scahs để lưu trữ file đầu vào là một chuỗi chứa một hoặc nhiều đường dẫn  được 
    # phân tahcs bởi 1 dấu phẩy 
    input_files = []
    for input_pattern in FLAGS.input_file.split(","):
        input_files.extend(tf.gfile.Glob(input_pattern))

    # Ghi nhật ký thông tin “*** Reading from input files ***” 
    # để báo hiệu bắt đầu đọc các tệp đầu vào. Vòng lặp for sẽ duyệt qua các 
    # phần tử trong danh sách input_files và ghi nhật ký thông tin về đường dẫn của mỗi tệp
    tf.logging.info("*** Reading from input files ***")
    for input_file in input_files:
        tf.logging.info("  %s", input_file)


    # tạo một đối tượng rng là một đối tượng random.random sử dụng để tạo ra một xác suấ ngẫu nhiên
    rng = random.Random(FLAGS.random_seed)
    # gọi hàm create_training_instances là một lớp để lưu trữ các thông tin về một mẫu huấn luyện
    instances = create_training_instances(
        input_files, tokenizer, FLAGS.max_seq_length, FLAGS.dupe_factor,
         FLAGS.short_seq_prob, FLAGS.masked_lm_prob, FLAGS.max_predictions_per_seq,
      rng)
    # Tạo một danh sách gọi là output_files, là một danh sách các đường dẫn đến các tệp đầu ra
    #  để lưu trữ dữ liệu huấn luyện. Tham số output_file là một chuỗi chứa một hoặc nhiều đường dẫn, 
    # được phân tách bằng dấu phẩy. Hàm split sẽ tách chuỗi này thành một danh sách các đường dẫn.
    output_files = FLAGS.output_file.split(",")
    tf.logging.info("*** Writing to output files ***")
    #  Vòng lặp for sẽ duyệt qua các phần tử trong danh sách output_files và ghi nhật ký thông tin về đường dẫn của mỗi tệp
    for output_file in output_files:
        tf.logging.info("  %s", output_file)

    #ghi cc
    write_instance_to_example_files(instances, tokenizer, FLAGS.max_seq_length,
                                  FLAGS.max_predictions_per_seq, output_files)


if __name__ == "__main__":
  flags.mark_flag_as_required("input_file")
  flags.mark_flag_as_required("output_file")
  flags.mark_flag_as_required("vocab_file")
  tf.app.run()